
### Structured output
Models can be requested to provide their response in a format matching a given schema. This is useful for ensuring the output can be easily parsed and used in subsequent processing. LangChain supports multiple schema types and methods for enforcing structured output.

### Pydantic
Pydantic models provide the richest feature set with field validation, descriptions, and nested structures.

In [2]:
import os 
from langchain.chat_models import init_chat_model

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
model = init_chat_model("groq:qwen/qwen3-32b")
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000014A46500D70>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000014A46501A90>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
from pydantic import BaseModel, Field

class Movie(BaseModel):
    title:str = Field(description="The title of the movie")
    year:int = Field(description="This year the movie was released")
    actors:str = Field(description="The actors of the movie")
    director:str = Field(description="The director of the movie")
    rating:float = Field(description="The movies rating out of 10")


In [9]:
model_with_structure = model.with_structured_output(Movie)
model_with_structure

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000014A46500D70>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000014A46501A90>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Movie', 'description': '', 'parameters': {'properties': {'title': {'description': 'The title of the movie', 'type': 'string'}, 'year': {'description': 'This year the movie was released', 'type': 'integer'}, 'actors': {'description': 'The actors of the movie', 'type': 'string'}, 'director': {'description': 'The director of the movie', 'type': 'string'}, 'rating': {'description': 'The movie

In [10]:
model_with_structure.invoke("Provide details about the movie Inception")

Movie(title='Inception', year=2010, actors='Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page, Tom Hardy', director='Christopher Nolan', rating=8.8)

### Message output alongside parsed structure

In [ ]:
class Movie(BaseModel):
    """ A movie with details."""
    title:str = Field(description="The title of the movie")
    year:int = Field(description="This year the movie was released")
    actors:str = Field(description="The actors of the movie")
    director:str = Field(description="The director of the movie")
    rating:float = Field(description="The movies rating out of 10")

In [11]:
### Nested Structure
model_with_structure = model.with_structured_output(Movie, include_raw=True)
response = model_with_structure.invoke("Provide details about the movie Inception")
response

{'raw': AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for details about the movie Inception. Let me see what I need to do. The available tool is the Movie function, which requires specific parameters: title, year, actors, director, and rating. I need to make sure I include all these details.\n\nFirst, the title is obviously "Inception". The release year was 2010. The main actors include Leonardo DiCaprio, Joseph Gordon-Levitt, Ellen Page, and Tom Hardy. The director is Christopher Nolan. The rating is a bit tricky, but I remember it\'s highly rated; maybe around 8.8 on IMDb. Let me confirm that. Yes, it\'s 8.8/10. \n\nWait, the parameters require the rating as a number. So I should put 8.8. Now, I need to structure this into the function call. Let me double-check each required field. Title, year, actors, director, rating. All there. I\'ll format the actors as a comma-separated string. Same for the director. Alright, that should cover it. Let me